In [1]:
from djitellopy import tello
import cv2
import HandTrackingModule as htm
import autopy #For controlling the mouse
import numpy as np
import time# Set resolution 
import os

In [2]:
gui_background = cv2.imread(r"\GUI\GUI1.png")
gui_background = cv2.resize(gui_background,(768,432),interpolation=cv2.INTER_AREA)
#Import mode png into a list
folderModepath = r"\GUI\Mode"
Mode_list = []
Mode = []
modePathList = os.listdir(folderModepath)
for path in modePathList:
    Mode_list.append(cv2.imread(os.path.join(folderModepath,path)))
    print(path)

for img in Mode_list:
    img = cv2.resize(img,(231,28),interpolation=cv2.INTER_AREA)
    Mode.append(img)

00Connected.png
01Control.png
02Follow.png
03Takeoff.png
04Landing.png
05Forward.png
06Back.png
07Left.png
08Right.png
09Up.png
10Down.png
11Rotate.png
12Flip.png


In [ ]:
# tello = tello.Tello()
# tello.connect()
# print(tello.get_battery())
# tello.streamon() #打开摄像头

In [6]:
wCam, hCam = 400, 225
cap = cv2.VideoCapture(0)  # Default camera on laptop: 0  External:1 or more
cap.set(3, wCam)  #Fix the w/h ratio to 3:4
cap.set(4, hCam) #and set the 分辨率

detector = htm.handDetector(maxHands = 1)
flag = [0,0,0,0,0]

In [ ]:
Mode[0].shape

(28, 231, 3)

In [ ]:
while True:
    success, img_origin = cap.read() #按帧读取视频
                              #如果读取值是正确帧，success=True,如果读取到结尾，False
                              #img就是每一帧的图像，是个三维矩阵.在cv2里是BGR
    img_origin = cv2.resize(img_origin,(wCam,hCam),interpolation=cv2.INTER_AREA)

    allhands,img_origin = detector.findHands(img_origin, draw=True) #手部定位
    
    # photo = tello.get_frame_read().frame
    # photo = cv2.resize(photo, (215, 145))
    gui_background[100:100+hCam,44:wCam-6] = img_origin[0:hCam,25:wCam-25]  #[starting point of hlast point of h,starting of wlast of w]
    # gui_background[4646+photo.shape[0],498498+photo.shape[1]] = photo  

    #cv2.rectangle(gui_background,(180,375),(218,395),(186,96,104),-1)
    # cv2.putText(gui_background, str(tello.get_battery()), (186,391), cv2.FONT_ITALIC,
    #                             0.75, (255, 255, 255), 1)

##############################################################################################################
    cv2.rectangle(gui_background,(180,375),(500,395),(186,96,104),-1)
    cv2.putText(gui_background, str(fingers), (186,391), cv2.FONT_ITALIC,0.75, (255, 255, 255), 1) #测试用
##############################################################################################################

    cv2.imshow("Drone control",gui_background)
    cv2.waitKey(1) #keep the window visible rather than shutting down instantly
    
    if len(allhands) != 0:
        x1, y1 = allhands[0]['lmList'][4][0:2]
        x2, y2 = allhands[0]['lmList'][8][0:2]
        x3, y3 = allhands[0]['lmList'][12][0:2]
        fingers = detector.fingersUp(img_origin, allhands)

        if fingers == [0,1,1,0,0] and flag[0] == 0: #take off
            gui_background[232:232+Mode[0].shape[0],490:490+Mode[0].shape[1]] = Mode[3]  #Display mode
            flag[0] = 1

        elif fingers == [-1,0,0,0,0] and flag[1] == 0: #landing
            gui_background[232:232+Mode[0].shape[0],490:490+Mode[0].shape[1]] = Mode[4]  #Display mode
            flag[1] = 1

        elif fingers == [0,1,0,0,0] and flag[2] == 0: #left
            gui_background[232:232+Mode[0].shape[0],490:490+Mode[0].shape[1]] = Mode[7]  #Display mode
            flag[2] = 1

        elif fingers == [0,0,0,0,1] and flag[3] == 0: #right
            gui_background[232:232+Mode[0].shape[0],490:490+Mode[0].shape[1]] = Mode[8]  #Display mode
            flag[3] = 1

        elif fingers == [0,0,0,0,0]:
            flag = [0,0,0,0,0]

#################################################################
    if cv2.waitKey(1) == ord('q'):
     break

cap.release()
cv2.destroyAllWindows()